## MPESA STATEMENT ANALYZER 

#### This is an AI model that gets an mpesa statement and automatically opens it with  approval from the user  and analyzes the pdf file  to come up with a beviour analysis of our users 

In [85]:
import PyPDF2 as py 
import fitz as f 
import io
import tabula
import pandas as pd
import spacy
import nltk


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

[nltk_data] Downloading package punkt to /home/kokim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kokim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/kokim/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/kokim/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [74]:

def pdf_decrypt(pdf_password, pdf_file):
    """A function that decrypts a PDF file and extracts tables using tabula-py."""
    
    # Open the PDF file
    with open(pdf_file, "rb") as file:
        pdf_reader = py.PdfReader(file)

        # Check if the PDF is encrypted
        if pdf_reader.is_encrypted:
            # Attempt to decrypt the PDF with the password
            if pdf_reader.decrypt(str(pdf_password)):
                print("PDF successfully decrypted.")
            else:
                raise ValueError("Failed to decrypt the PDF. Please check your password.")
        else:
            print("PDF is not encrypted.")
    
    # Extract tables from the PDF using tabula
    dfs = tabula.read_pdf(pdf_file,password=str(pdf_password),pages='all')

    new_table = dfs[2::2]
    #concatenating all the tables in the list into one dataframe
    mpesa_df = pd.concat(dfs,axis=0,ignore_index = True)
    #Filtering for only the completed transactions
    mpesa_df = mpesa_df[mpesa_df['Transaction\rStatus'] =='Completed']


    return mpesa_df # Return the extracted tables as DataFrames


# calling the function 
pdf_password = 37360412
pdf_file = "/home/kokim/projects/Esabu/MPESA_Statement_20190313_to_20190913_254713403607.pdf"


df = pdf_decrypt(pdf_password=pdf_password,pdf_file=pdf_file)

PDF successfully decrypted.


Got stderr: Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Nov 06, 2024 3:24:14 PM or

In [75]:
df.isna().sum()

TRANSACTION TYPE       1216
PAID IN                1216
PAID OUT               1216
Receipt No.               0
Completion Time           0
Details                   0
Unnamed: 0             1216
Transaction            1216
Paid In                 778
Withdrawn               438
Balance                   0
Transaction\rStatus       0
dtype: int64

#### Cleaning the pdf file 



In [76]:
def data_program(data):
    """
    A function that assists in data understanding.
    It returns the number of rows, columns, data types, missing values, and duplicates.
    """
    # Getting the number of missing values
    missing_val = data.isna().sum()

    # Getting the number of duplicates
    dup_val = data.duplicated().sum()

    # Getting the data types
    data_types = data.dtypes 

    # Getting the number of rows and columns
    rows, columns = data.shape 

    # Print a separator for readability
    print("......")
    
    # Returning formatted string with basic info
    return (f"The dataframe consists of {rows} rows and {columns} columns.\n"
            f"Data types:\n{data_types}\n\n"
            f"Missing values per column:\n{missing_val}\n\n"
            f"Total duplicates: {dup_val}")

# Example call to the function with a dataframe (assuming 'df' is defined)
print(data_program(df))



......
The dataframe consists of 1216 rows and 12 columns.
Data types:
TRANSACTION TYPE        object
PAID IN                 object
PAID OUT                object
Receipt No.             object
Completion Time         object
Details                 object
Unnamed: 0             float64
Transaction             object
Paid In                 object
Withdrawn               object
Balance                 object
Transaction\rStatus     object
dtype: object

Missing values per column:
TRANSACTION TYPE       1216
PAID IN                1216
PAID OUT               1216
Receipt No.               0
Completion Time           0
Details                   0
Unnamed: 0             1216
Transaction            1216
Paid In                 778
Withdrawn               438
Balance                   0
Transaction\rStatus       0
dtype: int64

Total duplicates: 0


In [77]:
df.head()

,TRANSACTION TYPE,PAID IN,PAID OUT,Receipt No.,Completion Time,Details,Unnamed: 0,Transaction,Paid In,Withdrawn,Balance,Transaction\rStatus
55,NaN,NaN,NaN,NI94GZ0JPE,2019-09-09 12:24:36,Customer Transfer to 254722794556 - PATRICK\rG...,NaN,NaN,NaN,"-2,500.00",521.39,Completed
56,NaN,NaN,NaN,NI91GYPE0H,2019-09-09 12:13:20,Funds received from 254728109253 - VINCENT\rKI...,NaN,NaN,"3,500.00",NaN,"3,500.00",Completed
57,NaN,NaN,NaN,NI92GYPG4M,2019-09-09 12:13:20,OD Loan Repayment to 232323 - M-PESA Overdraw,NaN,NaN,NaN,-478.61,"3,021.39",Completed
58,NaN,NaN,NaN,NI80GPKPBS,2019-09-08 22:33:02,OverDraft of Credit Party,NaN,NaN,57.00,NaN,2.00,Completed
59,NaN,NaN,NaN,NI80GPKPBS,2019-09-08 22:33:02,Pay Bill Charge,NaN,NaN,NaN,-2.00,0.00,Completed


In [78]:
df.columns

Index(['TRANSACTION TYPE', 'PAID IN', 'PAID OUT', 'Receipt No.',
       'Completion Time', 'Details', 'Unnamed: 0', 'Transaction', 'Paid In',
       'Withdrawn', 'Balance', 'Transaction\rStatus'],
      dtype='object')

In [79]:
def cleaning(data):
    """
    A function to clean Mpesa statements by converting object columns to float, 
    dropping columns with more than 50% missing values, and dropping duplicates.
    """

    # Remove commas from the 'Paid In', 'Withdrawn', and 'Balance' columns before conversion
    data["Paid In"] = data['Paid In'].str.replace(',', '').astype(float).fillna(0)
    data["Withdrawn"] = data['Withdrawn'].str.replace(',', '').astype(float).fillna(0)
    data["Balance"] = data['Balance'].str.replace(',', '').astype(float).fillna(0)
    # Remove '\r' from the 'Name' column
    data['Details'] = data['Details'].str.replace('\r',' ',regex=False)

    # converting the completion to datetime data type 
    data['Completion Time'] = pd.to_datetime(data['Completion Time'])


    # replaci

    # Dropping columns that have more than 50% missing values
    for column in data.columns:
        if data[column].isna().sum() / len(data) > 0.5:
            data.drop(column, axis=1, inplace=True)
        else:
            # Filling remaining missing values with the column mean (for numeric columns only)
            if pd.api.types.is_numeric_dtype(data[column]):
                data[column].fillna(data[column].mean(), inplace=True)

    # Dropping duplicates
    data.drop_duplicates(inplace=True)
    
    # Clean column names by removing any line breaks or unusual characters
    data.columns = data.columns.str.replace(r'[\r\n]', '', regex=True)

    # Now, drop the column
    data.drop(['TransactionStatus'], axis=1, inplace=True)  


    # converting the withdrawn data into  an abs value 
    data["Withdrawn"] = data["Withdrawn"].abs()



    return data

# Applying the cleaning function
df_cleaned = cleaning(df)
df_cleaned.head()



/tmp/ipykernel_147508/1411197818.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(data[column].mean(), inplace=True)


,Receipt No.,Completion Time,Details,Paid In,Withdrawn,Balance
55,NI94GZ0JPE,2019-09-09 12:24:36,Customer Transfer to 254722794556 - PATRICK GI...,0.0,2500.00,521.39
56,NI91GYPE0H,2019-09-09 12:13:20,Funds received from 254728109253 - VINCENT KIP...,3500.0,0.00,3500.00
57,NI92GYPG4M,2019-09-09 12:13:20,OD Loan Repayment to 232323 - M-PESA Overdraw,0.0,478.61,3021.39
58,NI80GPKPBS,2019-09-08 22:33:02,OverDraft of Credit Party,57.0,0.00,2.00
59,NI80GPKPBS,2019-09-08 22:33:02,Pay Bill Charge,0.0,2.00,0.00


###### Feature engineering 
Getting new columns from existing data 

In [80]:
def date_columns(data):
    """A function that gets the month and day of the week from the columns"""
    # getting the month from the completion time column
    data['month_name'] = data['Completion Time'].dt.month_name()
    
    # Getting the day name
    data['day_name'] = data['Completion Time'].dt.day_name()


    return data


df_cleaned = date_columns(df)
df.head()

,Receipt No.,Completion Time,Details,Paid In,Withdrawn,Balance,month_name,day_name
55,NI94GZ0JPE,2019-09-09 12:24:36,Customer Transfer to 254722794556 - PATRICK GI...,0.0,2500.00,521.39,September,Monday
56,NI91GYPE0H,2019-09-09 12:13:20,Funds received from 254728109253 - VINCENT KIP...,3500.0,0.00,3500.00,September,Monday
57,NI92GYPG4M,2019-09-09 12:13:20,OD Loan Repayment to 232323 - M-PESA Overdraw,0.0,478.61,3021.39,September,Monday
58,NI80GPKPBS,2019-09-08 22:33:02,OverDraft of Credit Party,57.0,0.00,2.00,September,Sunday
59,NI80GPKPBS,2019-09-08 22:33:02,Pay Bill Charge,0.0,2.00,0.00,September,Sunday


In [81]:
def map_categories(data):
    """Categorize 'Details' column entries and map them to cleaned, user-friendly categories."""
    
    # Define the mapping dictionary
    mapped_categories = {
        'Customer Transfer to': 'Send Money',
        'Pay Bill Fuliza M-Pesa to' : 'Fuliza Loan',
        'Customer Transfer Fuliza MPesa': 'Send Money',
        'Pay Bill Online': 'Pay Bill',
        'Pay Bill to': 'Pay Bill',
        'Customer Transfer of Funds\rCharge': 'Mpesa Charges',
        'Pay Bill Charge': 'Mpesa Charges',
        'Merchant Payment Online': 'Till No',
        'Customer Payment to Small': 'Pochi',
        'M-Shwari Withdraw': 'Mshwari Withdraw',
        'Business Payment from': 'Bank Transfer',
        'Airtime Purchase': 'Airtime Purchase',
        'Funds received from': 'Received Money',
        'Merchant Payment': 'Till No',
        'Customer Withdrawal': 'Cash Withdrawal',
        'Withdrawal Charge': 'Mpesa Charges',
        'Pay Merchant Charge': 'Mpesa Charges',
        'M-Shwari Deposit': 'Mshwari Deposit',
        'M-Shwari Loan': 'M-Shwari Loan',
        'Deposit of Funds at Agent': 'Customer Deposit',
        'OD Loan Repayment to': 'Fuliza Loan Repayment',
        'OverDraft of Credit Party': 'Fuliza Loan',
        'Customer Transfer Fuliza M-Pesa to':'Send Money',
        'Customer Transfer of Funds Charge':'Mpesa Charges',
        'KCB M-PESA Withdraw': 'KCB M-PESA Withdraw',
        'KCB M-PESA Deposit': 'KCB M-PESA Deposit',
        'KCB M-PESA Target Deposit': 'KCB M-PESA Deposit',	
        'Promotion Payment':'Received Money',
        'KCB M-PESA Target First Deposit':'KCB M-PESA Deposit',
        'Other': 'Other'
    }
    
    # Initialize the 'Category' column with a default value of 'Other'
    data['Category'] = 'Other'

    # Loop through each phrase in mapped_categories and assign the appropriate category
    for phrase, category in mapped_categories.items():
        data.loc[data['Details'].str.contains(phrase, case=False, na=False), 'Category'] = phrase

    # Map 'Category' to user-friendly names in 'Transaction_Type' column
    data['Transaction_Type'] = data['Category'].map(mapped_categories)

    
    

    return data


def drop_unwanted(data ):
    """A function that drops unwanted rows  that were created during mapping of the  categories"""    # Now, drop the column
    
    data.drop(['Category'], axis=1, inplace=True)  
    # Remove rows where 'Transaction_Type' is "Mpesa Charges"
    data = data.drop(data[data['Transaction_Type'] == "Mpesa Charges"].index)

    return data


    
# Use the function on your DataFrame
df_cleaned = map_categories(df)
df_cleaned =drop_unwanted(df)



In [82]:
df_cleaned.head()


,Receipt No.,Completion Time,Details,Paid In,Withdrawn,Balance,month_name,day_name,Transaction_Type
55,NI94GZ0JPE,2019-09-09 12:24:36,Customer Transfer to 254722794556 - PATRICK GI...,0.0,2500.00,521.39,September,Monday,Send Money
56,NI91GYPE0H,2019-09-09 12:13:20,Funds received from 254728109253 - VINCENT KIP...,3500.0,0.00,3500.00,September,Monday,Received Money
57,NI92GYPG4M,2019-09-09 12:13:20,OD Loan Repayment to 232323 - M-PESA Overdraw,0.0,478.61,3021.39,September,Monday,Fuliza Loan Repayment
58,NI80GPKPBS,2019-09-08 22:33:02,OverDraft of Credit Party,57.0,0.00,2.00,September,Sunday,Fuliza Loan
60,NI80GPKPBS,2019-09-08 22:33:02,Pay Bill Fuliza M-Pesa to 777711 - TELKOM KENY...,0.0,55.00,-55.00,September,Sunday,Fuliza Loan


### getting specific names 

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/kokim/nltk_data'
    - '/home/kokim/projects/kim_venv/nltk_data'
    - '/home/kokim/projects/kim_venv/share/nltk_data'
    - '/home/kokim/projects/kim_venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [90]:
import nltk
print(nltk.data.find('tokenizers/punkt'))
print(nltk.data.find('taggers/averaged_perceptron_tagger'))
print(nltk.data.find('chunkers/maxent_ne_chunker'))
print(nltk.data.find('corpora/words'))



/home/kokim/nltk_data/tokenizers/punkt
/home/kokim/nltk_data/taggers/averaged_perceptron_tagger
/home/kokim/nltk_data/chunkers/maxent_ne_chunker
/home/kokim/nltk_data/corpora/words
